<a href="https://colab.research.google.com/github/Sibaso/Project3_quan.nt173312/blob/main/Mult_NMT_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.5MB 22.6MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 
     |████████████████████████████████| 2.9MB 56.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=43edd319fddb81df2ab43d0bfd5c7919bc02cdcf6749c8d75bd112125a784612
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torchtext

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from collections import defaultdict
from copy import deepcopy
import numpy as np
import pandas as pd
import re
from torchtext.data import Field, BucketIterator, TabularDataset, Dataset, Example
from torchsummary import summary
import math
import copy
from copy import deepcopy
from torch.optim.optimizer import Optimizer
import os
import time
import nltk
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# device='cpu'

cuda


In [3]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# MultilingualBert = BertModel.from_pretrained("bert-base-multilingual-cased")

In [4]:

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.att_fc = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.att_fc(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)
        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.trb_fc1 = nn.Linear(embed_size, forward_expansion * embed_size)
        self.relu = nn.ReLU()
        self.trb_fc2 = nn.Linear(forward_expansion * embed_size, embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.trb_fc1(x)
        forward = self.relu(forward)
        forward = self.trb_fc2(forward)
        out = self.dropout(self.norm2(forward + x))
        return out


class Encoder(nn.Module):
    def __init__(
        self,
        embed_size,
        num_layers,
        forward_expansion,
        heads,
        dropout,
        vocab_size,
        max_length
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )
        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(
        self,
        embed_size,
        num_layers,
        forward_expansion,
        heads,
        dropout,
        vocab_size,
        max_length
    ):
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return F.log_softmax(out, dim=-1)


class Transformer(nn.Module):
    def __init__(
        self,
        embed_size=768,
        num_layers=1,
        forward_expansion=4,
        heads=12,
        dropout=0.1,
        vocab_size=119547,
        max_length=512,
        pad_idx=0
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            embed_size,
            num_layers,
            forward_expansion,
            heads,
            dropout,
            vocab_size,
            max_length
        )

        self.decoder = Decoder(
            embed_size,
            num_layers,
            forward_expansion,
            heads,
            dropout,
            vocab_size,
            max_length
        )

        self.pad_idx = pad_idx

    def make_src_mask(self, src):
        src_mask = (src != self.pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out


In [6]:

class Appr(object):
    """ Class implementing the Elastic Weight Consolidation approach described in http://arxiv.org/abs/1612.00796 """

    def __init__(self,model, lamb=4000, nepochs=100,sbatch=32,lr=0.001,lr_min=1e-6,lr_factor=3,lr_patience=5,name='model'):
        self.model=model
        self.fisher=None
        self.tasks = []

        self.nepochs = nepochs
        self.sbatch = sbatch
        self.lr = lr
        self.lr_min = lr_min
        self.lr_factor = lr_factor
        self.lr_patience = lr_patience
        self.lamb = lamb

        self.name = name


    def add_task(self, src_token, trg_token):
        if {'src': src_token, 'trg': trg_token} in self.tasks:
            return
        self.tasks.append({'src': src_token, 'trg': trg_token})
        
    def load_task(self, t):
        path = '/content/drive/My Drive/Project3/trained_model/{}|{}'.format(self.name,
                        '|'.join(['%s-%s'%(task['src'], task['trg']) for task in self.tasks[:t]]))
        self.model.load_state_dict(torch.load('%s.pt'%path))
        self.model.to(device)
        # self.fisher = torch.load('%s_fisher.pt'%path)

    def get_param(self, model):
        sd = []
        for name, param in model.named_parameters():
            if 'word_embedding' in name or 'position_embedding' in name or 'norm' in name or 'norm1' in name or 'norm2' in name:
                continue 
            sd.append((name, param))
        return sd

    def get_optimizer(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)
        # return torch.optim.SGD(self.model.parameters(), lr=self.lr, momentum=0.9)
   
    def train(self, src_train, trg_train, src_valid, trg_valid):
        best_loss = np.inf
        lr = self.lr
        patience = self.lr_patience
        self.ce=torch.nn.CrossEntropyLoss(ignore_index=self.model.pad_idx)
        self.optimizer=self.get_optimizer()
        path = '/content/drive/My Drive/Project3/trained_model/{}|{}'.format(self.name,
                        '|'.join(['%s-%s'%(task['src'], task['trg']) for task in self.tasks]))
        # Loop epochs
        for e in range(self.nepochs):
            try:
                # Train
                clock0=time.time()
                train_loss = self.train_epoch(src_train, trg_train)
                clock1=time.time()
                valid_loss = self.eval(src_valid, trg_valid)
                clock2=time.time()
                print('| Epoch {:3d}, time={:5.1f}ms/{:5.1f}ms | Train: loss={:.3f} |'.format(
                    e+1,1000*(clock1-clock0),
                    1000*(clock2-clock1),train_loss),end='')
                # # Valid
                print(' Valid: loss={:.3f} |'.format(valid_loss),end='')
                
                # Adapt lr
                if valid_loss < best_loss:
                    best_loss = valid_loss
                    torch.save(self.model.state_dict(),'%s.pt' % path)
                    patience = self.lr_patience
                    print(' *', end='')
                
                else:
                    patience -= 1
                    if patience <= 0:
                        lr /= self.lr_factor
                        print(' lr={:.1e}'.format(lr), end='')
                        if lr < self.lr_min:
                            print()
                            break
                            
                        patience = self.lr_patience
                        self.optimizer = self.get_optimizer()
                print()
            except KeyboardInterrupt:
                break

        # Restore best
        # torch.save(self.model.state_dict(),'%s.pt' % path)
        print('loading best model ...')
        self.model.load_state_dict(torch.load('%s.pt'%path))
        return

    def train_epoch(self, src_data, trg_data):
        self.model.train()
        total_loss=0
        idx = np.random.permutation(src_data.shape[0])
        num_iters = src_data.shape[0]//self.sbatch+1
        for i in range(num_iters):
            top = min((i+1)*self.sbatch, src_data.shape[0])
            src = src_data[idx[i*self.sbatch:top]].to(device)
            trg = trg_data[idx[i*self.sbatch:top]].to(device)
            output = self.model(src, trg[:, :-1])
            output = output.reshape(-1, output.shape[2])
            trg = trg[:,1:].reshape(-1)   
            loss=self.ce(output,trg)
            total_loss += loss.item()
            # Backward
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        return total_loss/num_iters

    def eval(self, src_data, trg_data):
        total_loss=0
        self.model.eval()
        idx = np.random.permutation(src_data.shape[0])
        num_iters = src_data.shape[0]//self.sbatch+1
        for i in range(num_iters):
            top = min((i+1)*self.sbatch, src_data.shape[0])
            src = src_data[idx[i*self.sbatch:top]].to(device)
            trg = trg_data[idx[i*self.sbatch:top]].to(device)
            output = self.model(src, trg[:, :-1])
            output = output.reshape(-1, output.shape[2])
            trg = trg[:,1:].reshape(-1)   
            loss=self.ce(output,trg)
            total_loss += loss.item()

        return total_loss/num_iters


def translate(model, src_sen, max_length=512):

    model.eval()
    src_tensor = tokenizer(src_sen, return_tensors='pt')['input_ids'].to(device)
    outputs = [tokenizer.cls_token_id]
    for i in range(max_length):
        translate_tensor = torch.LongTensor(outputs).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(src_tensor, translate_tensor)
        
        best_guess = output.argmax(2)[:,-1].item()
        outputs.append(best_guess)

        if best_guess == tokenizer.sep_token_id:
            break

    outputs = outputs[1:-1]
    translated_sentence = tokenizer.decode(outputs)
    return translated_sentence

def eval_bleu_score(model, src_data, trg_data):
    total_bleu = 0
    for src_sen, trg_sen in zip(src_data, trg_data):
        translated_sentence = translate(model, src_sen)
        bleu_score = nltk.translate.bleu_score.sentence_bleu([trg_sen.split()], translated_sentence.split(), weights=(0.5, 0.5))
        total_bleu += bleu_score
    return total_bleu/len(src_data)

In [20]:
model = Transformer(
    embed_size=768,
    num_layers=1,
    forward_expansion=4,
    heads=12,
    dropout=0.1,
    vocab_size=tokenizer.vocab_size,
    max_length=512,
    pad_idx=0
).to(device)

appr = Appr(
    model=model,
    lamb=4000,
    nepochs=20,
    sbatch=32,
    lr=0.001,
    lr_min=1e-6,
    lr_factor=3,
    lr_patience=5,
    name='mult_bert_single'
)

In [21]:
def preprocess(text):
    # text = ViTokenizer.tokenize(text)
    return ' '.join(re.split('\W+', text)).lower()

def load_data(path):
    with open('/content/drive/MyDrive/Project3/data/%s'%path, encoding='utf-8') as f:
        data = [line.split('\t')[:2] for line in f.read().splitlines()]

    src = [preprocess(line[0]) for line in data]
    trg = [preprocess(line[1]) for line in data]
    return src, trg

src_train, trg_train = [], []
src_valid, trg_valid = [], []
src_test, trg_test = [], []

# train data
# eng_train, viet_train = load_data('eng-viet_train.txt')
# src_train += viet_train
# trg_train += eng_train

# eng1_train, fra_train = load_data('eng-fra_train.txt')
# eng1_train, fra_train = eng1_train[:5000], fra_train[:5000]
# src_train += fra_train
# trg_train += eng1_train

# eng2_train, deu_train = load_data('eng-deu_train.txt')
# eng2_train, deu_train = eng2_train[:5000], deu_train[:5000]
# src_train += deu_train
# trg_train += eng2_train

eng3_train, cmn_train = load_data('eng-cmn_train.txt')
eng3_train, cmn_train = eng3_train[:5000], cmn_train[:5000]
src_train += cmn_train
trg_train += eng3_train


# valid data
# eng_valid, viet_valid = load_data('eng-viet_valid.txt')
# src_valid += viet_valid
# trg_valid += eng_valid

# eng1_valid, fra_valid = load_data('eng-fra_valid.txt')
# eng1_valid, fra_valid = eng1_valid[:600], fra_valid[:600]
# src_valid += fra_valid
# trg_valid += eng1_valid

# eng2_valid, deu_valid = load_data('eng-deu_valid.txt')
# eng2_valid, deu_valid = eng2_valid[:600], deu_valid[:600]
# src_valid += deu_valid
# trg_valid += eng2_valid

eng3_valid, cmn_valid = load_data('eng-cmn_valid.txt')
eng3_valid, cmn_valid = eng3_valid[:600], cmn_valid[:600]
src_valid += cmn_valid
trg_valid += eng3_valid


# test data
# eng_test, viet_test = load_data('eng-viet_test.txt')
# src_test += viet_test
# trg_test += eng_test

# eng1_test, fra_test = load_data('eng-fra_test.txt')
# eng1_test, fra_test = eng1_test[:700], fra_test[:700]
# src_test += fra_test
# trg_test += eng1_test

# eng2_test, deu_test = load_data('eng-deu_test.txt')
# eng2_test, deu_test = eng2_test[:700], deu_test[:700]
# src_test += deu_test
# trg_test += eng2_test

eng3_test, cmn_test = load_data('eng-cmn_test.txt')
eng3_test, cmn_test = eng3_test[:700], cmn_test[:700]
src_test += cmn_test
trg_test += eng3_test


src_train_tensor = tokenizer(src_train, return_tensors='pt', padding=True)['input_ids']
trg_train_tensor = tokenizer(trg_train, return_tensors='pt', padding=True)['input_ids']

src_valid_tensor = tokenizer(src_valid, return_tensors='pt', padding=True)['input_ids']
trg_valid_tensor = tokenizer(trg_valid, return_tensors='pt', padding=True)['input_ids']

print('train:', len(src_train))
print('valid:', len(src_valid))
print('test:', len(src_test))

train: 5000
valid: 600
test: 700


In [22]:
# appr.add_task('viet|fra|deu|cmn','eng')
appr.train(src_train_tensor, trg_train_tensor, src_valid_tensor, trg_valid_tensor)

| Epoch   1, time=45075.6ms/1481.0ms | Train: loss=5.667 | Valid: loss=4.706 | *
| Epoch   2, time=45048.8ms/1438.5ms | Train: loss=3.871 | Valid: loss=4.167 | *
| Epoch   3, time=45253.1ms/1426.5ms | Train: loss=2.905 | Valid: loss=3.898 | *
| Epoch   4, time=45281.1ms/1437.0ms | Train: loss=2.128 | Valid: loss=3.865 | *
| Epoch   5, time=45220.9ms/1434.6ms | Train: loss=1.508 | Valid: loss=3.924 |
| Epoch   6, time=45025.6ms/1438.0ms | Train: loss=1.048 | Valid: loss=4.105 |
| Epoch   7, time=45096.2ms/1433.6ms | Train: loss=0.739 | Valid: loss=4.265 |
loading best model ...


In [23]:
# task1_bleu = eval_bleu_score(model, viet_test, eng_test)
# print('task 1, test viet-eng:', task1_bleu)

# task2_bleu = eval_bleu_score(model, fra_test, eng1_test)
# print('task 2, test fra-eng:', task2_bleu)

# task3_bleu = eval_bleu_score(model, deu_test, eng2_test)
# print('task 3, test deu-eng:', task3_bleu)

task4_bleu = eval_bleu_score(model, cmn_test, eng3_test)
print('task 4, test cmn-eng:', task4_bleu)



/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 4, test cmn-eng: 0.3870527406688129
